In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import DBSCAN
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def fetch_data(filename):
    """
    This function is used to upload training file and convert into dataframe.
    :param: filename, :type: .csv
    :return: df
    :return type: class 'pandas.core.frame.DataFrame'
    """
    path = '../input/'+filename+'.csv'
    df = pd.read_csv(path)
    return df

In [ ]:
file = "ccdata/CC GENERAL"
file2 = "iris/Iris"
file3 = "headbrain/headbrain"
df = fetch_data(file)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
if "Species" in df.columns:
    df = df.drop("Species",axis=1)
    df = df.drop("Id",axis=1)
elif "CUST_ID" in df.columns:
    df = df.drop("CUST_ID",axis=1)
else:
    pass

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
df=scaler.transform(df)

In [ ]:
df

In [ ]:
"""
def fit_dbscan(eps,samples,max_amonaly_percentage):
    dbscan = DBSCAN(eps=eps, min_samples=samples)
    result = dbscan.fit(df)
    print(result.labels_)
    labels = result.labels_ 
    print(np.unique(labels))
    labelsdf = pd.DataFrame(labels)
    vals = labelsdf.value_counts()
    print(vals)
    print("iteration complete")#First clusters are created with user defined eps,minsamples
    for i in vals.index:
        print("cluster",int(i[0])," counts",vals[i])
        count = vals[i]
        cluster = int(i[0])
        if cluster == -1:
            if count>((labels.size/100)*max_amonaly_percentage):
                #If anomaly size is greater than max allowed, then increase eps to reduce anomaly
                eps = eps+0.2
                #If some clusters strictly follwing minsamples as 2, then remove them by increasing samples 
                if vals.min() <= 2:
                    samples = samples+1
            
                labels, eps, samples, vals = fit_dbscan(eps,samples,max_amonaly_percentage)
                break
    return labels, eps, samples, vals
"""

In [ ]:
#eps = 1.0
#samples = 2
#max_amonaly_percentage = 5 #How much percent of anomaly should be allowed in clusters, reduces values in cluster -1
#labels, eps, samples, vals = fit_dbscan(eps,samples,max_amonaly_percentage)

In [ ]:
#labelsdf2 = pd.DataFrame(labels)
#print("Final EPS: ",eps," Final Min-samples: ",samples)
#labelsdf2.value_counts()

In [ ]:
def fit_dbscan2(eps,samples,anomaly_count2):
    dbscan = DBSCAN(eps=eps, min_samples=samples)
    result = dbscan.fit(df)
    labels = result.labels_ 
    print(np.unique(labels))
    labelsdf = pd.DataFrame(labels)
    vals = labelsdf.value_counts()
    print(vals)
    print("iteration complete")#First clusters are created with user defined eps,minsamples
    anomaly_count = vals[-1]
    print("anomaly_count ",anomaly_count)
    print("anomaly_count2 ",anomaly_count2)
    if anomaly_count2 == 0:
        eps=eps+0.05
        labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count)
    elif (anomaly_count2-anomaly_count)>1: #Compare anomaly counts of n and n-1 iteration
        anomaly_count2=anomaly_count
        eps=eps+0.05
        labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count2)
    else:
        return labels, eps, samples, vals
    return labels, eps, samples, vals

In [ ]:
eps = 1.0
samples = 2
anomaly_count2=0
labels, eps, samples, vals = fit_dbscan2(eps,samples,anomaly_count2)

In [ ]:
labelsdf2 = pd.DataFrame(labels)
print("Final EPS: ",eps," Final Min-samples: ",samples)
labelsdf2.value_counts()